In [1]:
!pip install confluent-kafka==1.7.0

     |████████████████████████████████| 2.7 MB 11.9 MB/s eta 0:00:01


In [3]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4
import random

In [4]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [5]:
conf = {'bootstrap.servers': bootstrap_server}

In [6]:
a = AdminClient(conf)

In [7]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 2 topics:
  "_schemas" with 1 partition(s)
  "ParticularAdEvent" with 1 partition(s)


In [8]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *

import time

topic = "ClickEvent"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [9]:
producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')
}

producer = SerializingProducer(producer_conf)

### Run it

The assumption is the following if the user has been generated the event, it means that he has clicked on it.
Also, the sleeping time is random since the user is the real human

In [10]:
import json
from IPython.display import clear_output

keys = ["A", "B", "C", "D", "F", "G"]

i = 0

while True:
    if i == len(keys):
        i = 0
    key = keys[i]
    value = {"choose_ad": key, "ts":int(time.time())}
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    i += 1
    time.sleep(random.randint(9, 40))  # NOTICE: here upper bound is 40, which means that we will wait a little bit long
                                       # but this number has been chosen due to being useful for outer joins
    clear_output(wait=True)

{'choose_ad': 'C', 'ts': 1650970392}
Produced record to topic ClickEvent partition [0] @ offset 37


KeyboardInterrupt: 